### Ingestion del archivo "movie_genre.json"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

### Paso 1 - Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
movies_genres_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("genreId", IntegerType(), True)
])

In [0]:
movies_genres_df = spark.read \
                   .schema(movies_genres_schema) \
                   .json(f"{bronze_folder_path}/{v_file_date}/movie_genre.json")

### Paso 2 - Renombrar las columnas y añadir nuevas columnas
1. "movieId" renombrar a "movie_id"
2. "genreId" renombrar a "genre_id"
3. Agregar las columnas "ingestion_date" y "environment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
movies_genre_final_df = add_ingestion_date(movies_genres_df) \
                        .withColumn("environment", lit(v_environment)) \
                        .withColumn("file_date", lit(v_file_date)) \
                        .withColumnRenamed("movieId","movie_id") \
                        .withColumnRenamed("genreId","genre_id")
                        

In [0]:
display(movies_genre_final_df)

### Paso 3 - Escribir la salida en formato "Parquet"

In [0]:
#overwrite_partition(movies_genre_final_df,"movie_silver","movies_genres","file_date")

In [0]:
# movies_genre_final_df.write.mode("overwrite").partitionBy("movie_id").parquet(f"{silver_folder_path}/movies_genres")

# Crear una tabla en base al data frame movies_genre_final_df
# Creara la tabla movies_genres en a DB que se llame movie_silver
# Modulo: Spark SQL - Database / Table / View: seccion: Crear Tabla - Capa Silver
#movies_genre_final_df.write.mode("overwrite").partitionBy("movie_id").format("parquet").saveAsTable("movie_silver.movies_genres")

# Append, agrega los registros a la tabla existente, carga incremental de datos
#movies_genre_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.movies_genres")

#Modulo: Delta Lake
merge_condition = "tgt.movie_id = src.movie_id AND tgt.genre_id = src.genre_id AND tgt.file_date = src.file_date" # Buscar por particion
merge_delta_lake(movies_genre_final_df,"movie_silver","movies_genres",silver_folder_path,merge_condition,"file_date")

In [0]:
%sql
-- Consultar la DB movie_silver, tabla movies_genres
SELECT file_date, COUNT(1)
FROM movie_silver.movies_genres
GROUP BY 1;

In [0]:
#display(spark.read.parquet(f"{silver_folder_path}/movies_genres"))

In [0]:
dbutils.notebook.exit("Success")